In [1]:
import numpy as np
import cvxpy as cp
import wta

np.set_printoptions(precision=3, suppress=True, linewidth=200)

In [2]:
# Data
n = 2
tgts = 3
wpns = 2
m = (tgts, wpns)
# Survival probabilities
Q = np.array([[0.8, 0.4],
              [0.6, 0.5],
              [0.4, 0.6]])

# Target values
V = np.array([10, 6, 10])

# Weapon values
WW = np.array([1, 1])

In [4]:
p, x = wta.wta(Q, V, WW)


optimal
The optimal value is 13.999999957685631
A solution x is
[[-0.  1.]
 [ 0.  0.]
 [ 1.  0.]]


In [5]:
wta.wta(Q, V, WW, integer=False)

optimal
The optimal value is 14.18056618827489
A solution x is
[[-0.     0.921]
 [ 0.     0.079]
 [ 1.    -0.   ]]


(14.18056618827489,
 array([[-0.   ,  0.921],
        [ 0.   ,  0.079],
        [ 1.   , -0.   ]]))

In [ ]:
# Set gamma, L, W, itrs, v0

L = np.zeros((n,n))
L[1,0] = 2

W = np.ones((n,n))
W[1,0] = -1
W[0,1] = -1

itrs = 2
gamma = 0.5

v0 = []
vk = []
for i in range(n):
    v0.append(np.zeros(m))
    vk.append(np.zeros(m))

node_tgts = {0:[0], 1:[1,2]}
num_nodes_per_tgt = [1, 1, 1]

In [ ]:
# Create variables/params/objs for the algorithm
probs = [] # List of problems for each node
all_x = [] # List of last x solutions for each node as params
all_v = [] # List of last v solutions for each node as params
all_w = []# List of variables for each node
for i in range(n):
    w = cp.Variable(m)
    all_w.append(w)
    x = cp.Parameter(m)
    x.value = np.zeros(m)
    all_x.append(x)
    v = cp.Parameter(m)
    v.value = v0[i]
    all_v.append(v)
    y = v + sum(L[i,j]*all_x[j] for j in range(i)) + L[i,i]*w
    q = np.ones(m)
    q[node_tgts[i]] = Q[node_tgts[i]] # Only use the targets that are in the node
    weighted_weapons = cp.multiply(w, np.log(q)) # (tgts, wpns)
    survival_probs = cp.exp(cp.sum(weighted_weapons, axis=1)) # (tgts,)
    VV = np.zeros(tgts)
    VV[node_tgts[i]] = V[node_tgts[i]]
    obj = cp.Minimize(VV@survival_probs + .5*cp.sum_squares(w - y))
    cons = [w >= 0, cp.sum(w, axis=0) <= WW]
    probs.append(cp.Problem(obj, cons))

In [ ]:
for itr in range(10):
    print("Iteration", itr)
    for i in range(n):
        print("Node", i)
        probs[i].solve()
        print("w", all_w[i].value)
        all_x[i].value = all_w[i].value
    for i in range(n):
        vk[i] -= gamma*sum(W[i,j]*all_x[j].value for j in range(n))
        all_v[i].value = vk[i]
    print("v", vk)


Iteration 0
Node 0
w [[0.754 1.   ]
 [0.    0.   ]
 [0.    0.   ]]
Node 1
w [[0.    0.177]
 [0.    0.713]
 [1.    0.109]]
v [array([[-0.377, -0.411],
       [-0.   ,  0.357],
       [ 0.5  ,  0.055]]), array([[ 0.377,  0.411],
       [ 0.   , -0.357],
       [-0.5  , -0.055]])]
Iteration 1
Node 0
w [[0.433 1.   ]
 [0.    0.   ]
 [0.5   0.   ]]
Node 1
w [[0.    0.448]
 [0.    0.54 ]
 [1.    0.012]]
v [array([[-0.594, -0.688],
       [-0.   ,  0.627],
       [ 0.75 ,  0.061]]), array([[ 0.594,  0.688],
       [ 0.   , -0.627],
       [-0.75 , -0.061]])]
Iteration 2
Node 0
w [[0.25 1.  ]
 [0.   0.  ]
 [0.75 0.  ]]
Node 1
w [[0.    0.591]
 [0.    0.409]
 [1.    0.   ]]
v [array([[-0.719, -0.892],
       [-0.   ,  0.831],
       [ 0.875,  0.061]]), array([[ 0.719,  0.892],
       [ 0.   , -0.831],
       [-0.875, -0.061]])]
Iteration 3
Node 0
w [[0.136 1.   ]
 [0.    0.   ]
 [0.864 0.   ]]
Node 1
w [[0.    0.687]
 [0.    0.313]
 [1.    0.   ]]
v [array([[-0.787, -1.048],
       [-0.   ,  0.

In [ ]:
Q

array([[0.8, 0.4],
       [0.6, 0.5],
       [0.4, 0.6]])

In [ ]:
Q[[0,2],:]

array([[0.8, 0.4],
       [0.4, 0.6]])